# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
import hvplot.pandas
import pandas as pd
import requests

# Load the city data CSV
city_data_df = pd.read_csv("output_data/cities.csv")

# Configure the map plot with world map and dots overlayed for cities
cities_map = city_data_df.hvplot.points(
    'Lng', 'Lat',           # Use Longitude and Latitude for the points
    color='City',           # Color by city
    size='Humidity',        # Size by humidity
    hover_cols=['City', 'Country', 'Humidity'],  # Display city, country, and humidity on hover
    title="City Locations and Humidity",
    frame_width=800,        # Width of the plot
    frame_height=600,       # Height of the plot
    tiles='OSM'             # This adds the OpenStreetMap as the background (you can also use other tile sources)
)

# Display the map
cities_map




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity,Country)

In [9]:
# Narrow down cities that fit criteria
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
152,kahului,US,20.8947,-156.4700,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel (10 km radius)
radius = 10000  # in meters
params = {
    'apiKey': geoapify_key,  # API key
    'radius': radius,        # search radius in meters
    'limit': 1,              # limit to 1 result per search
    'type': 'hotel'          # type of location
}

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params['lat'] = latitude
    params['lon'] = longitude

    # Set the base URL for the Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request using the parameters
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found."
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search...
kapaa - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
kahului - nearest hotel: No hotel found
gat - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: No hotel found
santa rosalia - nearest hotel: No hotel found
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel found
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
capoterra - nearest hotel: No hotel found
stolac - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,No hotel found
51,hilo,US,19.7297,-155.0900,83,No hotel found
63,banda,IN,25.4833,80.3333,52,No hotel found
81,makakilo city,US,21.3469,-158.0858,81,No hotel found
152,kahului,US,20.8947,-156.4700,60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
# Configure the map plot to include hotel names in hover message
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',            # Use Longitude and Latitude for the points
    color='City',            # Color the points by city name
    size='Humidity',         # Size the points by humidity
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],  # Show city, country, humidity, and hotel name on hover
    title="City Locations, Humidity, and Nearest Hotels",  # Set the correct title
    frame_width=800,         # Width of the plot
    frame_height=600,        # Height of the plot
    tiles='OSM'              # Use OpenStreetMap as the background
)

# Set axis labels (Longitude for x-axis, Latitude for y-axis)
hotel_map.opts(
    xlabel='Longitude',       # Label for x-axis
    ylabel='Latitude'         # Label for y-axis
)

# Display the map
hotel_map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity,Country,Hotel Name)